In [ ]:
# install isimip-client from GitHub
!pip install git+https://github.com/ISI-MIP/isimip-client
# install dependecies for plotting
!pip install netCDF4 matplotlib pandas

In [ ]:
from isimip_client.client import ISIMIPClient
client = ISIMIPClient()

In [ ]:
# search the ISIMIP Repository for the datasets
response = client.datasets(simulation_round='ISIMIP3b',
                           product='InputData',
                           climate_forcing='gfdl-esm4',
                           climate_scenario='ssp126',
                           climate_variable='sfcwind')

file_paths = []
for dataset in response["results"]:
    for file in  dataset['files']:
        file_paths.append(file['path'])

In [ ]:
points = [
    (52.518611, 13.408333, 'downloads/berlin'),
    (40.712778, -74.005833, 'downloads/new-york-city'),
    (39.906667, 116.3975, 'downloads/beijing'),
    (-23.5, -46.616667, 'downloads/sao-paulo'),
    (-4.331667, 15.313889, 'downloads/kinshasa')
]

In [ ]:
for lat, lon, download_path in points:
    response = client.select(file_paths, point=(lat, lon), poll=10)
    client.download(response['file_url'], path=download_path, validate=False, extract=True)

In [ ]:
# plot the time series
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

fig, axes = plt.subplots(nrows=len(points))
for index, point in enumerate(points):
    lat, lon, download_path = point
    frames = []
    for file in sorted(Path(download_path).iterdir()):
        if file.suffix == '.csv':
            frames.append(pd.read_csv(file, names=['date', 'sfcwind'], index_col='date', parse_dates=True))
    
    df = pd.concat(frames)
    df.plot(ax=axes[index], title=download_path, xlim=('2020-01-01', '2030-01-01'))

plt.gcf().set_size_inches(20, 40)